### Import Libraries

In [1]:

import numpy as np
import pandas as pd
import glob
import time
import pandas as pd
# from xml.dom import minidom

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords 


import os
import matplotlib.pyplot as plt
import keras

import cv2

[nltk_data] Downloading package punkt to /home/ebryx/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ebryx/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ebryx/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Using TensorFlow backend.


### Read Benign data
Source :
1. scrapy.txt. https://docs.scrapy.org/en/latest/
2. wikipedia.txt https://en.wikipedia.org/wiki/Artificial_intelligence

In [2]:
path='./data/wikipedea.txt'

wikipedia=[]
f = open(path, "r")
for x in f:
    wikipedia.append(x)

In [3]:
# remove blank lines
benign_wikipedia=[]
for x in wikipedia:
    if x == "\n":
        pass
    else:
        line=x[:-1]
        benign_wikipedia.append(line)

In [4]:
benign_wikipedia[:5]

['<!DOCTYPE html>',
 '<html class="client-nojs" lang="en" dir="ltr">',
 '<head>',
 '<meta charset="UTF-8"/>',
 '<title>Artificial intelligence - Wikipedia </title>']

In [5]:
len(benign_wikipedia)

5539

In [6]:


path='./data/scrapy.txt'

scrapy=[]
f = open(path, "r")
for x in f:
    scrapy.append(x)

In [7]:
# remove blank lines
benign_scrapy=[]
for x in scrapy:
    if x == "\n":
        pass
    else:
        line=x[:-1]
        benign_scrapy.append(line)

In [8]:
benign_scrapy[:5]

['<!DOCTYPE html>',
 '<html class="no-js" lang="en" >',
 '<head>',
 '  <meta charset="utf-8">',
 '  <meta name="viewport" content="width=device-width, initial-scale=1.0">']

In [9]:
len(benign_scrapy)

774

### Read XSS data
Source :
1. owasp.txt. https://owasp.org/www-community/xss-filter-evasion-cheatsheet
2. portswigger.txt https://portswigger.net/web-security/cross-site-scripting/cheat-sheet


In [10]:

path='./data/owasp.txt'

owasp=[]
f = open(path, "r")
for x in f:
    owasp.append(x)

In [11]:
# remove blank lines
xss_owasp=[]
for x in owasp:
    if x == "\n":
        pass
    else:
        line=x[:-1]
        xss_owasp.append(line)

In [12]:
xss_owasp[:5]

['javascript:/*--></title></style></textarea></script></xmp><svg/onload=\'+/"/+/onmouseover=1/+/[*/[]/+alert(1)//\'>',
 '<IMG SRC="javascript:alert(\'\');">',
 "<IMG SRC=javascript:alert('')>",
 "<IMG SRC=JaVaScRiPt:alert('')>",
 '<IMG SRC=javascript:alert(&quot;&quot;)>']

In [13]:
len(xss_owasp)

252

In [14]:

path='./data/portswigger.txt'

portswigger=[]
f = open(path, "r")
for x in f:
    portswigger.append(x)

In [15]:
# remove blank lines
xss_portswigger=[]
for x in portswigger:
    if x == "\n":
        pass
    else:
        line=x[:-1]
        xss_portswigger.append(line)

In [16]:
xss_portswigger[:5]

['<a draggable="true" ondrag="alert(1)">test</a>',
 '<a draggable="true" ondragend="alert(1)">test</a>',
 '<a draggable="true" ondragenter="alert(1)">test</a>',
 '<a draggable="true" ondragleave="alert(1)">test</a>',
 '<a draggable="true" ondragstart="alert(1)">test</a>']

In [17]:
print(len(xss_portswigger))

7121


In [18]:
print("Total XSS attacks : ", len(xss_owasp)+len(xss_portswigger))

Total XSS attacks :  7373


In [19]:
print("Total benign data : ", len(benign_scrapy)+len(benign_wikipedia))

Total benign data :  6313


#### Combine data

In [20]:
# Combine benign data and give label of 0

data=[]
for i in benign_wikipedia:
    data.append((i,0))

In [21]:
for i in benign_scrapy:
    data.append((i,0))

In [22]:
# Combine XSS data and give label of 1

for i in xss_owasp:
    data.append((i,1))

In [23]:
for i in xss_portswigger:
    data.append((i,1))

In [24]:
print("Total data : ", len(data))

Total data :  13686


### Save data as CSV

In [25]:
df=pd.DataFrame(data,columns=['Sentence','Label'])

In [26]:
df.tail()

,Sentence,Label
13681,<xss id=x tabindex=1 onblur=alert(1)></xss><in...,1
13682,<xss id=x tabindex=1 ondeactivate=alert(1)></x...,1
13683,<xss id=x tabindex=1 onfocus=alert(1)></xss>,1
13684,<xss id=x tabindex=1 onfocusin=alert(1)></xss>,1
13685,<xss id=x tabindex=1 onfocusout=alert(1)></xss...,1


In [27]:
df.tail()

,Sentence,Label
13681,<xss id=x tabindex=1 onblur=alert(1)></xss><in...,1
13682,<xss id=x tabindex=1 ondeactivate=alert(1)></x...,1
13683,<xss id=x tabindex=1 onfocus=alert(1)></xss>,1
13684,<xss id=x tabindex=1 onfocusin=alert(1)></xss>,1
13685,<xss id=x tabindex=1 onfocusout=alert(1)></xss...,1


In [28]:
# shuffle data and reset index
df = df.sample(frac=1).reset_index(drop=True)

In [29]:
df.to_csv('./data/XSS_dataset.csv', encoding='utf-8-sig')

In [30]:
df=pd.read_csv('./data/XSS_dataset.csv', encoding='utf-8-sig')

In [31]:
df.head()

,Unnamed: 0,Sentence,Label
0,0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0
1,1,"<tt onmouseover=""alert(1)"">test</tt>",1
2,2,"\t </span> <span class=""reference-text"">Steeri...",0
3,3,"\t </span> <span class=""reference-text""><cite ...",0
4,4,"\t </span>. <a href=""/wiki/Digital_object_iden...",0


In [32]:
df=df[df.columns[-2:]]  # Only get sentence and labels

In [33]:
df.head()

,Sentence,Label
0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0
1,"<tt onmouseover=""alert(1)"">test</tt>",1
2,"\t </span> <span class=""reference-text"">Steeri...",0
3,"\t </span> <span class=""reference-text""><cite ...",0
4,"\t </span>. <a href=""/wiki/Digital_object_iden...",0


In [34]:
# Get Sentences data from data frame
sentences=df['Sentence'].values
sentences[1]

'<tt onmouseover="alert(1)">test</tt>'

In [35]:
print(len(sentences))

13686


In [191]:
# Convert to ASCII

def convert_to_ascii(sentence):
    sentence_ascii=[]

    for i in sentence:
        
        
        """Some characters have values very big e.d 8221 adn some are chinese letters
        I am removing letters having values greater than 8222 and for rest greater 
        than 128 and smaller than 8222 assigning them values so they can easily be normalized"""
       
        if(ord(i)<8222):      # ” has ASCII of 8221
            
            if(ord(i)==8217): # ’  :  8217
                sentence_ascii.append(134)
            
            
            if(ord(i)==8221): # ”  :  8221
                sentence_ascii.append(129)
                
            if(ord(i)==8220): # “  :  8220
                sentence_ascii.append(130)
                
                
            if(ord(i)==8216): # ‘  :  8216
                sentence_ascii.append(131)
                
            if(ord(i)==8217): # ’  :  8217
                sentence_ascii.append(132)
            
            if(ord(i)==8211): # –  :  8211
                sentence_ascii.append(133)
                
                
            """
            If values less than 128 store them else discard them
            """
            if (ord(i)<=128):
                    sentence_ascii.append(ord(i))
    
            else:
                    pass
            

    zer=np.zeros((10000))

    for i in range(len(sentence_ascii)):
        zer[i]=sentence_ascii[i]

    zer.shape=(100, 100)


#     plt.plot(image)
#     plt.show()
    return zer


In [192]:


# send each sentence to be converted to ASCII


arr=np.zeros((len(sentences),100,100))

for i in range(len(sentences)):
    
    image=convert_to_ascii(sentences[i])

    x=np.asarray(image,dtype='float')
    image =  cv2.resize(x, dsize=(100,100), interpolation=cv2.INTER_CUBIC)
    image/=128

    
#     if i==1:
#         plt.plot(image)
#         plt.show()    
    arr[i]=image



In [193]:
print("Input data shape : ", arr.shape)

Input data shape :  (13686, 100, 100)


In [194]:
# Reshape data for input to CNN
data = arr.reshape(arr.shape[0], 100, 100, 1)

In [195]:
data.shape

(13686, 100, 100, 1)

In [196]:
y=df['Label'].values

In [197]:
# Split into train/test data
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(data,y, test_size=0.2, random_state=42)

In [159]:
# import libraries for Making Model
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D,Flatten,Dropout,MaxPool2D, BatchNormalization

In [160]:
# A basic CNN Model
# Number of layers = 11
# Number of Convolutional layer: 3

model=tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(64,(3,3), activation=tf.nn.relu, input_shape=(100,100,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(256,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    
])

In [161]:

model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 98, 98, 64)        640       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 47, 47, 128)       73856     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 23, 23, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 21, 21, 256)       295168    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 10, 10, 256)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 25600)            

In [162]:
# Stop when validation accuracy > 97

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.97):
            print(" \n Reached 97% + validation accuracy")
            self.model.stop_training=True
callbacks = myCallback()

In [163]:
batch_size = 128
num_epoch = 10
#model training
model_log = model.fit(trainX, trainY,
          batch_size=batch_size,
          epochs=num_epoch,
          verbose=1,
          validation_data=( testX,  testY)
#                        callbacks=[callbacks]
                     )

Train on 10948 samples, validate on 2738 samples
Epoch 1/10
10948/10948 [==============================] - 121s 11ms/sample - loss: 0.4544 - accuracy: 0.7789 - val_loss: 0.3946 - val_accuracy: 0.8083
Epoch 2/10
10948/10948 [==============================] - 128s 12ms/sample - loss: 0.2203 - accuracy: 0.9185 - val_loss: 0.0808 - val_accuracy: 0.9759
Epoch 3/10
10948/10948 [==============================] - 124s 11ms/sample - loss: 0.0624 - accuracy: 0.9831 - val_loss: 0.0652 - val_accuracy: 0.9843
Epoch 4/10
10948/10948 [==============================] - 117s 11ms/sample - loss: 0.0576 - accuracy: 0.9817 - val_loss: 0.0625 - val_accuracy: 0.9814
Epoch 5/10
10948/10948 [==============================] - 118s 11ms/sample - loss: 0.0424 - accuracy: 0.9870 - val_loss: 0.0433 - val_accuracy: 0.9879
Epoch 6/10
10948/10948 [==============================] - 117s 11ms/sample - loss: 0.0429 - accuracy: 0.9884 - val_loss: 0.0606 - val_accuracy: 0.9795
Epoch 7/10
10948/10948 [=====================

In [164]:
# predict for test set
pred=model.predict(testX)


In [165]:
# Threshold values predicted

for i in range(len(pred)):
    if pred[i]>0.5:
        pred[i]=1
    elif pred[i]<=0.5:
        pred[i]=0



In [166]:
# Count True predicted and wrong predicted

true=0
false=0

for i in range(len(pred)):
    if pred[i] == testY[i]:
        true+=1
    else:
        false+=1
        
print("correct predicted :: ", true)
print("false prediction :: ", false)

correct predicted ::  2712
false prediction ::  26


In [167]:
# Number of attack and benign data in test set

attack=0
benign=0
for i in range(len(testY)):
    if testY[i]==1:
        attack+=1
    else:
        benign+=1

print("Attack data in test set :: ", attack)
print(" Benign data in test set :: ", benign)

Attack data in test set ::  1478
 Benign data in test set ::  1260


### Confusion Matrix

In [168]:
# Takes True positive, true negative, false positive and false negative as input
# Returns Accuracy

def accuracy_function(tp,tn,fp,fn):
    
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    
    return accuracy

In [169]:
# Takes True Positive and false positive
# Returns Precision

def precision_function(tp,fp):
    
    precision = tp / (tp+fp)
    
    return precision

In [170]:
#Takes True Positive and false negative
# Returns Recall

def recall_function(tp,fn):
    
    recall=tp / (tp+fn)
    
    return recall

In [171]:
#Takes true values and predicted
# Returns Accuracy, precision and recall


def confusion_matrix(truth,predicted):
    
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    
    for true,pred in zip(truth,predicted):
        if true == 1:
            if pred == true:
                true_positive += 1
            elif pred != true:
                false_negative += 1

        elif true == 0:
            if pred == true:
                true_negative += 1
            elif pred != true:
                false_positive += 1
            
    accuracy=accuracy_function(true_positive, true_negative, false_positive, false_negative)
    precision=precision_function(true_positive, false_positive)
    recall=recall_function(true_positive, false_negative)
    
    return (accuracy,
            precision,
           recall)

In [172]:
accuracy,precision,recall=confusion_matrix(testY,pred)

In [173]:
print(" Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

 Accuracy : 0.9905040175310446 
 Precision : 0.9852941176470589 
 Recall : 0.9972936400541272


In [174]:
from sklearn.metrics import precision_score
precision_score(testY, pred)

0.9852941176470589

In [175]:

from sklearn.metrics import recall_score
recall_score(testY, pred)

0.9972936400541272

### Save Model

In [176]:

from keras.models import load_model

model.save('my_model_cnn.h5')

In [178]:
from keras.models import load_model


mymodel = tf.keras.models.load_model('my_model_cnn.h5')

### See prediction on data

In [187]:
# Convert to ASCII

def convert_to_ascii(sentence):
    sentence_ascii=[]

    for i in sentence:
        
        
        """Some characters have values very big e.d 8221 adn some are chinese letters
        I am removing letters having values greater than 8222 and for rest greater 
        than 128 and smaller than 8222 assigning them values so they can easily be normalized"""
       
        if(ord(i)<8222):      # ” has ASCII of 8221
            
            if(ord(i)==8217): # ’  :  8217
                sentence_ascii.append(134)
            
            
            if(ord(i)==8221): # ”  :  8221
                sentence_ascii.append(129)
                
            if(ord(i)==8220): # “  :  8220
                sentence_ascii.append(130)
                
                
            if(ord(i)==8216): # ‘  :  8216
                sentence_ascii.append(131)
                
            if(ord(i)==8217): # ’  :  8217
                sentence_ascii.append(132)
            
            if(ord(i)==8211): # –  :  8211
                sentence_ascii.append(133)
                
                
            """
            If values less than 128 store them else discard them
            """
            if (ord(i)<=128):
                    sentence_ascii.append(ord(i))
    
            else:
                    pass
            

    zer=np.zeros((10000))

    for i in range(len(sentence_ascii)):
        zer[i]=sentence_ascii[i]

    zer.shape=(100, 100)


#     plt.plot(image)
#     plt.show()
    return zer


In [188]:
import numpy as np
import cv2









def predict_cross_site_script():
    
    repeat=True
    
    
    
    
    beautify=''
    for i in range(20):
        beautify+= "="

    print(beautify) 
    input_val=input("Give me some data to work on : ")
    print(beautify)
    
    if input_val== '0':
        repeat=False
        
        
    
    if repeat == True:

        image=convert_to_ascii(input_val)

        x=np.asarray(image,dtype='float')
        image =  cv2.resize(x, dsize=(100,100), interpolation=cv2.INTER_CUBIC)
        image.shape=(1,100,100,1)
        image/=128

        prediction=mymodel.predict(image)

        if prediction>0.5:

    #         print(f"Chances of attack :  {prediction[0]*100} ")
            print(" It can be Cross site scripting attack")

        else:

    #         print(f" Chances of being safe {100 - (prediction*100) }")
            print("seems safe")
        
        
        predict_cross_site_script()
        
        
    elif repeat == False:
        print( " Good Bye ")

In [190]:
predict_cross_site_script()

Give me some data to work on : <IMG SRC="jav	ascript:alert('');">
 It can be Cross site scripting attack
Give me some data to work on : <IMG SRC=JaVaScRiPt:alert('')>
 It can be Cross site scripting attack
Give me some data to work on : <INPUT TYPE="IMAGE" SRC="javascript:alert('');">
 It can be Cross site scripting attack
Give me some data to work on : <STYLE>.{background-image:url("javascript:alert('')");}</STYLE><A CLASS=></A>
 It can be Cross site scripting attack
Give me some data to work on : SRC=# onmouseover="alert(document.cookie)
 It can be Cross site scripting attack
Give me some data to work on : tYPE="text/x-scriptlet" DATA="http://.rocks/scriptlet.html
 It can be Cross site scripting attack
Give me some data to work on : <?xml:namespace prefix="t" ns="urn:schemas-microsoft-com:time"> <?import namespace="t" implementation="#default#time2"> <t:set attributeName="innerHTML" to="<SCRIPT DEFER>alert("")</SCRIPT>">
 It can be Cross site scripting attack
Give me some data to wor

 It can be Cross site scripting attack
Give me some data to work on : <summary onfocusout=alert(1) tabindex=1 id=x></summary><input autofocus>
 It can be Cross site scripting attack
Give me some data to work on : <sub id=x tabindex=1 onbeforedeactivate=alert(1)></sub><input autofocus>
 It can be Cross site scripting attack
Give me some data to work on : <style>@keyframes x{}</style><wbr style="animation-name:x" onanimationend="alert(1)"></wbr>
 It can be Cross site scripting attack
Give me some data to work on : <style>@keyframes x{}</style><tt style="animation-name:x" onanimationstart="alert(1)"></tt>
 It can be Cross site scripting attack
Give me some data to work on : <style>@keyframes slidein {}</style><keygen style="animation-duration:1s;animation-name:slidein;animation-iteration-count:2" onanimationiteration="alert(1)"></keygen>
 It can be Cross site scripting attack
Give me some data to work on : <style onmousedown="alert(1)">test</style>
 It can be Cross site scripting attack
G

 It can be Cross site scripting attack
Give me some data to work on : <noembed onkeydown="alert(1)" contenteditable>test</noembed>
 It can be Cross site scripting attack
Give me some data to work on : <nobr onkeyup="alert(1)" contenteditable>test</nobr>
 It can be Cross site scripting attack
Give me some data to work on : <nextid onmousemove="alert(1)">test</nextid>
 It can be Cross site scripting attack
Give me some data to work on : <nextid id=x tabindex=1 onbeforedeactivate=alert(1)></nextid><input autofocus>
 It can be Cross site scripting attack
Give me some data to work on : <nav onfocusout=alert(1) tabindex=1 id=x></nav><input autofocus>
 It can be Cross site scripting attack
Give me some data to work on : <multicol onkeydown="alert(1)" contenteditable>test</multicol>
 It can be Cross site scripting attack
Give me some data to work on : <multicol id=x tabindex=1 onbeforedeactivate=alert(1)></multicol><input autofocus>
 It can be Cross site scripting attack
Give me some data to w

Give me some data to work on : <img id=x tabindex=1 ondeactivate=alert(1)></img><input id=y autofocus>
 It can be Cross site scripting attack
Give me some data to work on : <image2 onpointerout=alert(1)>XSS</image2>
 It can be Cross site scripting attack
Give me some data to work on : <image id=x tabindex=1 ondeactivate=alert(1)></image><input id=y autofocus>
 It can be Cross site scripting attack
Give me some data to work on : <iframe onkeyup="alert(1)" contenteditable>test</iframe>
 It can be Cross site scripting attack
Give me some data to work on : <iframe draggable="true" ondragenter="alert(1)">test</iframe>
 It can be Cross site scripting attack
Give me some data to work on : <i onfocusout=alert(1) tabindex=1 id=x></i><input autofocus>
 It can be Cross site scripting attack
Give me some data to work on : <html onmouseout="alert(1)">test</html>
 It can be Cross site scripting attack
Give me some data to work on : <html onkeyup="alert(1)" contenteditable>test</html>
 It can be Cros

 It can be Cross site scripting attack
Give me some data to work on : <dl onpaste="alert(1)" contenteditable>test</dl>
 It can be Cross site scripting attack
Give me some data to work on : <dl onkeydown="alert(1)" contenteditable>test</dl>
 It can be Cross site scripting attack
Give me some data to work on : <dl id=x tabindex=1 ondeactivate=alert(1)></dl><input id=y autofocus>
 It can be Cross site scripting attack
Give me some data to work on : <div onpointerout=alert(1)>XSS</div>
 It can be Cross site scripting attack
Give me some data to work on : <div onkeydown="alert(1)" contenteditable>test</div>
 It can be Cross site scripting attack
Give me some data to work on : <div draggable="true" contenteditable>drag me</div><ul ondrop=alert(1) contenteditable>drop here</ul>
 It can be Cross site scripting attack
Give me some data to work on : <div draggable="true" contenteditable>drag me</div><tbody ondrop=alert(1) contenteditable>drop here</tbody>
 It can be Cross site scripting attack
G

 It can be Cross site scripting attack
Give me some data to work on : <custom tags onpointerout=alert(1)>XSS</custom tags>
 It can be Cross site scripting attack
Give me some data to work on : <data draggable="true" ondragend="alert(1)">test</data>
 It can be Cross site scripting attack
Give me some data to work on : <content onkeyup="alert(1)" contenteditable>test</content>
 It can be Cross site scripting attack
Give me some data to work on : <content draggable="true" ondragstart="alert(1)">test</content>
 It can be Cross site scripting attack
Give me some data to work on : <content id=x tabindex=1 onbeforedeactivate=alert(1)></content><input autofocus>
 It can be Cross site scripting attack
Give me some data to work on : <command onmouseover="alert(1)">test</command>
 It can be Cross site scripting attack
Give me some data to work on : <command onmouseenter="alert(1)">test</command>
 It can be Cross site scripting attack
Give me some data to work on : <command onkeydown="alert(1)" co

 It can be Cross site scripting attack
Give me some data to work on : <body onkeypress="alert(1)" contenteditable>test</body>
 It can be Cross site scripting attack
Give me some data to work on : <body id=x tabindex=1 ondeactivate=alert(1)></body><input id=y autofocus>
 It can be Cross site scripting attack
Give me some data to work on : <body draggable="true" ondragend="alert(1)">test</body>
 It can be Cross site scripting attack
Give me some data to work on : <blockquote onkeydown="alert(1)" contenteditable>test</blockquote>
 It can be Cross site scripting attack
Give me some data to work on : <blockquote onbeforepaste="alert(1)" contenteditable>test</blockquote>
 It can be Cross site scripting attack
Give me some data to work on : <blockquote id=x tabindex=1 onbeforedeactivate=alert(1)></blockquote><input autofocus>
 It can be Cross site scripting attack
Give me some data to work on : <blink onpointerout=alert(1)>XSS</blink>
 It can be Cross site scripting attack
Give me some data t